In [1]:
import sys
sys.path.append('../src')
from order_book import Book
from event import Event

from scipy.linalg import block_diag
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import numpy as np
import plotly.graph_objects as go

import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

# import lobster message file
cols = ['time', 'type', 'id', 'shares', 'price', 'direction']
data = pd.read_csv("../data/lobster/MSFT_2012-06-21_34200000_37800000_message_50.csv", names=cols)
# re-scale the price col
data.price = data.price/10000
# make sure data is during market hours
data = data[data['time']>= 9.5*60*60]
data = data[data['time']<= 16*60*60]

print(len(data))
data.head()

141507


,time,type,id,shares,price,direction
0,34200.013994,3,16085616,100,31.04,-1
1,34200.013994,1,16116348,100,31.05,-1
2,34200.015248,1,16116658,100,31.04,-1
3,34200.015442,1,16116704,100,31.05,-1
4,34200.015789,1,16116752,100,31.06,-1


In [2]:
# read the messages and transform into an oderbook
book = Book()

for i in range(5_100):
    event = Event(data.loc[i])
    book.handleEvent(event, i)

In [3]:
# format the orderbook 
order_book = book.formatBook(start_from=100, levels=1)
print(len(order_book))
order_book.head()

5000


,Ask_1,Ask_1_Vol,Ask_1_Ord,Bid_1,Bid_1_Vol,Bid_1_Ord,Time
0,31.0,6700.0,2.0,30.95,300.0,1.0,03:30:00.197858
1,31.0,6700.0,2.0,30.95,300.0,1.0,03:30:00.197889
2,31.0,6700.0,2.0,30.95,300.0,1.0,03:30:00.197889
3,31.0,6700.0,2.0,30.95,300.0,1.0,03:30:00.197919
4,31.0,6700.0,2.0,30.95,300.0,1.0,03:30:00.197919


Micro Price Definition:

The micro price is the expected value of the mid price in the future
$$
    P^{micro}_{t} = \lim_{n -> \infty} P^{n}_{t}
$$

This can be approximated as a martingale prices
$$
    P^{n}_{t} = \exp[ M_{\tau_{n}} | I_{t}, S_{t}]
$$

Meaning that the micro price at time t is the expected value of the mid price at some random future time tau given the values of the imbalance I and spread S at time t

To model this, 2 assumptions are made

1. that the information in the book is given by the mid price M, the imbalance I, and the spread S
$$
    \mathcal{F_t} = \sigma({M_t}, {I_t}, {S_t})
$$
2. that the expected change in the mid price at the random future time tau, is only conditional on the imbalance I and the spread S
$$
    \exp[ M_{\tau_{i}} - M_{\tau_{i-1}} | M_{t} = M, I_{t} = I, S_{t} = S] = \exp[ M_{\tau_{i}} - M_{\tau_{i-1}} | I_{t} = I, S_{t} = S] , t <= \tau_{i-1}
$$

Start by calculating the state variables

1. Mid Price
$$
    M_{t} = \frac{1}{2} ( {P^b_t} + {P^a_t} )
$$
2. Imbalance
$$
    I_{t} = \frac{Q^b_t}{{Q^b_t} + {Q^a_t}}
$$
3. Spread
$$
    S_{t} = \frac{1}{2} ( {P^a_t} + {P^b_t} )
$$

We'll add the weighted mid price too 
$$
    Wmid_{t} = I_{t}{P^a_t} + (1 - I)_{t}{P^b_t}
$$

In [4]:
order_book['Mid'] = 0.5 * (order_book['Bid_1'] + order_book['Ask_1'])
order_book['Imbalance'] = order_book['Bid_1_Vol'] / (order_book['Bid_1_Vol'] + order_book['Ask_1_Vol'])
order_book['Spread'] = (order_book['Ask_1'] - order_book['Bid_1'])
order_book['Weighted_Mid'] = (order_book['Imbalance']*order_book['Ask_1']) + ((1-order_book['Imbalance'])*order_book['Bid_1'])
order_book.head()

,Ask_1,Ask_1_Vol,Ask_1_Ord,Bid_1,Bid_1_Vol,Bid_1_Ord,Time,Mid,Imbalance,Spread,Weighted_Mid
0,31.0,6700.0,2.0,30.95,300.0,1.0,03:30:00.197858,30.975,0.042857,0.05,30.952143
1,31.0,6700.0,2.0,30.95,300.0,1.0,03:30:00.197889,30.975,0.042857,0.05,30.952143
2,31.0,6700.0,2.0,30.95,300.0,1.0,03:30:00.197889,30.975,0.042857,0.05,30.952143
3,31.0,6700.0,2.0,30.95,300.0,1.0,03:30:00.197919,30.975,0.042857,0.05,30.952143
4,31.0,6700.0,2.0,30.95,300.0,1.0,03:30:00.197919,30.975,0.042857,0.05,30.952143


In [5]:
# plot the price traces
bid = go.Scatter(x=order_book.index, y=order_book.Bid_1, mode='lines', line=dict(color='green'), name='Bid')
ask = go.Scatter(x=order_book.index, y=order_book.Ask_1, mode='lines', line=dict(color='red'), name='Ask')
mid = go.Scatter(x=order_book.index, y=order_book.Mid, mode='lines', line=dict(color='blue'), name='Mid')
wmid = go.Scatter(x=order_book.index, y=order_book.Weighted_Mid, mode='lines', line=dict(color='black'), name='W-Mid')
traces = [bid, ask, mid, wmid]
fig = go.Figure(data=traces)
fig.update_layout(title='MSFT Bid / Ask', xaxis_title='Time', yaxis_title='Price', width=1440, height=800, hovermode='x')
#fig.show()

!['Price'](../data/images/prices.png)

In order to model the micro price, we have to make the state space discrete so the imbalance, spread, and change in mid price must be finite

1. Imbalance: this is continuous so we will group it into n buckets 
$$
    1 <= i_{I} <= n
$$
2. Spread: already discrete given it is defined in terms of tick size, so we will group by number of ticks m
$$
    1 <= i_{S} <= m
$$
3. Mid Price Change: this can only be 4 values given by K
$$
    M_{t+1} - M_{t}
$$
$$
    K = [-0.01, -0.005, 0.005, 0.01]^{T}
$$

So the state will be X:
$$
    X_{t} = (I_{t}, S_{t})
$$
With discrete values
$$
    1 <= i <= nm
$$

In [6]:
# 1. Starting with imbalance, we will bucket the values into n buckets
n_imbalance = 10
order_book['Imbalance_Bucket'] = pd.qcut(order_book['Imbalance'], n_imbalance, labels=False)

In [7]:
# see the different sizes of spread we have in the data set
spreads = go.Histogram(histfunc='count', x=order_book['Spread'], name='Spread Histogram')
fig = go.Figure(data=spreads)
fig.update_layout(title='Spread Counts', xaxis_title='Spread Size', yaxis_title='Count')

!['Spreads'](../data/images/spreads.png)

In [8]:
# 2. Then for spread, we'll get the tick size of the data set and then express the spread in terms of ticksize
tick_size = np.round(min(order_book['Spread']*100))/100
n_spread = 2
order_book = order_book.loc[(order_book['Spread'] <= n_spread*tick_size) & order_book['Spread'] > 0]
order_book['Spread'] = [np.round(x*100)/100 for x in order_book['Spread']]

In [9]:
# 3. finaly, for the change in mid price, delta mid, we'll take the time delta of T to T+1  and shift the values to fint the diff
dt = 1
order_book['Next_Mid'] = order_book['Mid'].shift(-dt)
order_book['Next_Spread'] = order_book['Spread'].shift(-dt)
order_book['Next_Time'] = order_book['Time'].shift(-dt)
order_book['Next_Imbalance_Bucket'] = order_book['Imbalance_Bucket'].shift(-dt)
# define the change in terms of tick size
order_book['Delta_Mid'] = np.round((order_book['Next_Mid'] - order_book['Mid'])/tick_size*2)*tick_size/2
# to get our values of K, we will limit the deltas to take only the values +/- 1 or +/- 0.5 of a tick
order_book = order_book.loc[(order_book['Delta_Mid'] <= tick_size*1.1) & (order_book['Delta_Mid'] >= -tick_size*1.1)]

In [10]:
# show discrete values:
print("n imbalance: ", set(order_book['Imbalance_Bucket']))
print("m spread: ", set(order_book['Spread']))
print("K Delta Mid: ", set(order_book['Delta_Mid']))

n imbalance:  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
m spread:  {0.02, 0.01}
K Delta Mid:  {0.0, -0.005, 0.01, -0.01, 0.005}


We symetrize the data now in order to estimate the micro price

Here wi will get:
$$
    I^{2}_{t} = n - I_{t}
$$
$$
    S^{2}_{t} = S_{t}
$$
$$
    ( M^{2}_{t + 1} - M^{2}_{t} ) = - ( M_{t + 1} - M_{t})
$$

In [11]:
# symetarize the data
order_book_sym = order_book.copy(deep=True)
order_book_sym['Imbalance_Bucket'] = n_imbalance - 1 -order_book_sym['Imbalance_Bucket']
order_book_sym['Next_Imbalance_Bucket'] = n_imbalance - 1 - order_book_sym['Next_Imbalance_Bucket']
order_book_sym['Delta_Mid'] = -order_book_sym['Delta_Mid']
order_book_sym['Mid'] = -order_book_sym['Mid']

symmetrized_data = pd.concat([order_book, order_book_sym])
symmetrized_data.index = pd.RangeIndex(len(symmetrized_data.index))

The micro price martingale approximation can be practically calculated with the formula

$$
    P^{n}_{t} = M_{t} + \sum^{n}_{k = 1} g^{K} (I_{t}, S_{t})
$$

Where the function g is a "micro price adjustment". The first adjustment, g1 is given as:

$$
    g^{1}(I_{t}, S_{t}) = exp[ M_{\tau_{1}} - M_{t} | I_{t}, S_{t}]
$$

So the micro price at time t is the mid at time t plus the expected change in the mid (i.e. the difference between the mid at time t and the mid at some time in the future, Tau) given the imbalance I and spread S at time t

We can restate g1 in terms of our markov state space X

$$
    g^{1}(i) = exp[ M_{\tau_{1}} - M_{t} | X_{t} = i]
$$

or, defined in terms of the actual transition matricies 

$$
    g^{1}(i) = (1 - Q)^{-1} R^{1} K
$$
Here we have 3 terms:
1. 'Transient States' = Q
    - This is a matrix of transition probabilities in the case where the mid price does not change but our imbalance or spread does change
$$
    Q_{ij} = prob( M_{t+1} - M_{t} = 0 ∧ X_{t+1} = j | X_{t} = i)
$$
2. 'Absorbing States' = R
    - this is a matrix of transition probabilities in the case where the mid price does change.
$$
    R^{1}_{iK} = prob( M_{t+1} - M_{t} = k | X_{t} = i)
$$

3. Possible price changes = K
    - different values a change in mid can take
$$
    k = [-0.01, -0.005, 0.005, 0.01]^{T}
$$

We start by estimating Q and R1 to get G1

In [12]:
# to estimate Q, we'll create an nm x nm matrix, which will give us every combination of spread size and imbalance bucket
# it will show us the transition probabilities from a given spread and imbalnce combination at time t to another at time t+1

# our symetarized data contains both Q and R values so we will filter for 0 change in mid price to find relevant Q values
no_change = symmetrized_data[symmetrized_data['Delta_Mid']==0]
# Now count the occurences for any given change of I in time t to I in time t+1
no_change_counts = no_change.pivot_table(index=['Next_Imbalance_Bucket'], columns=['Spread', 'Imbalance_Bucket'], values='Time', fill_value=0, aggfunc='count').unstack()
# turn into matrix of size n x n
q_counts = np.resize(np.array(no_change_counts[0:(n_imbalance*n_imbalance)]), (n_imbalance, n_imbalance))
# reshape the matrix to make it size nm x nm
for i in range(1, n_spread):
    qi = np.resize(np.array(no_change_counts[(i*n_imbalance*n_imbalance):(i+1)*(n_imbalance*n_imbalance)]), (n_imbalance, n_imbalance))
    q_counts= block_diag(q_counts,qi)
# taking row 1 as an example, this shows us the frequency of times that imbalance and spread change at time t + 1
# when at t, the imbalance = 0.1 and spread = 0.1 (becasue it it index 0)
q_counts[0]

array([451,  23,   9,   0,   2,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0])

In [13]:
# Now find the same frequency counts for R1
# filter for changes in mid prce
change = symmetrized_data[(symmetrized_data['Delta_Mid']!=0)]
# count occurences of mid changes at any given I and S
change_counts = change.pivot_table(index=['Delta_Mid'], columns=['Spread', 'Imbalance_Bucket'], values='Time', fill_value=0, aggfunc='count').unstack()
# reshape to an nm * 4 matrix
r_counts = np.resize(np.array(change_counts), (n_imbalance*n_spread, 4))
# row 1 as an example, this shows us the frequency of times that the mid chages in any of the 4 possible ways at t+1
# when at time t, the imbalance = 0.1 and spread = 0.1 (becasue it it index 0)
r_counts[0]

array([ 1, 36,  1,  0])

In [14]:
# Taking both Q and R, we can estimate the transition probabilites by dividing transition observations in any state
# by the total number of observations in our sample 
q_and_r_counts = np.concatenate((q_counts, r_counts), axis=1).astype(float)
for i in range(0, n_imbalance*n_spread):
    q_and_r_counts[i] = q_and_r_counts[i]/q_and_r_counts[i].sum()

Q = q_and_r_counts[:, 0:(n_imbalance*n_spread)]
R1 = q_and_r_counts[:,(n_imbalance*n_spread):]

# again taking index 0, we see the transition probabilities for a given change in t + 1 
# given at t, the imbalance = 0.1 and spread = 0.1
print("Transient States: ", Q[0])
print("Absorbing States: ", R1[0])

Transient States:  [0.8623327  0.04397706 0.01720841 0.         0.00382409 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.        ]
Absorbing States:  [0.00191205 0.06883365 0.00191205 0.        ]


In order to find the i-th micro price, we perform a recursive sum

(side note: this part I'm a little unclear on)

First, we define a new set of absorbing states, R2 (T in the paper):

$$
    R^{2}_{ik} = prob( M_{t+1} - M_{t} != 0 ∧ I_{t+1} = k | I_{t} = i)
$$

As above, we'll estimate R2

In [15]:
change = symmetrized_data[(symmetrized_data['Delta_Mid']!=0)]
change_counts = change.pivot_table(index=['Spread', 'Imbalance_Bucket'], columns=['Next_Spread', 'Next_Imbalance_Bucket'], values='Time', fill_value=0, aggfunc='count')
r2_counts = np.resize(np.array(change_counts), (n_imbalance*n_spread, n_imbalance*n_spread))
q_and_r2_counts = np.concatenate((q_counts, r2_counts), axis=1).astype(float)
for i in range(0, n_imbalance*n_spread):
    q_and_r2_counts[i] = q_and_r2_counts[i]/q_and_r2_counts[i].sum()
R2 = q_and_r2_counts[:,(n_imbalance*n_spread):]

Now that we have Q, R1, and R2, we can compute G

We start by computing the first order micro price adjustment, G1 
$$
    g^{1}(i) = (1 - Q)^{-1} R^{1} K
$$

And B
$$
    B = ( 1 - Q )^{-1} R^{2}
$$

In [16]:
K = np.array([-0.01, -0.005, 0.005, 0.01])
G1 = np.dot(np.dot(np.linalg.inv(np.eye(n_imbalance*n_spread)-Q),R1),K)
B = np.dot(np.linalg.inv(np.eye(n_imbalance*n_spread)-Q),R2)


Then, we can compute the i-th order micro price adjustment - Stoikov use i = 6
$$
    G^{*} = P^{micro} - M = G^{1} + \sum_{i=1}^{\infty} B^{i} G^{1}
$$

In [17]:
G2=np.dot(B,G1)+G1
G3=G2+np.dot(np.dot(B,B),G1)
G4=G3+np.dot(np.dot(np.dot(B,B),B),G1)
G5=G4+np.dot(np.dot(np.dot(np.dot(B,B),B),B),G1)
G6=G5+np.dot(np.dot(np.dot(np.dot(np.dot(B,B),B),B),B),G1)

In [24]:
imb = [x for x in range(n_imbalance)]
mid_price = np.linspace(-0.005, 0.005, n_imbalance)*0
weighted_mid = np.linspace(-0.005, 0.005, n_imbalance)
G6_spread_1 = G6[:n_imbalance]
G6_spread_2 = G6[n_imbalance:]

fig = go.Figure()
fig.add_trace(go.Scatter(x=imb, y=mid_price, name='Mid Price'))
fig.add_trace(go.Scatter(x=imb, y=weighted_mid, name='Weighted Mid'))
fig.add_trace(go.Scatter(x=imb, y=G6_spread_1, name='G6: Spread = 1 tick adj'))
fig.add_trace(go.Scatter(x=imb, y=G6_spread_2, name='G6: Spread = 2 tick adj'))
fig.update_layout(title='Adjustments', xaxis_title='Imbalance Bucket', yaxis_title='Price Change', height=700, width=800)
fig.show()

Here we plot the Mid and the weighted mid as functions of imbalance i.e. the change in the mid is unaffected by imbalance, and the change in the weighted mid changes linearly with imbalance

Similar to Stoikov's result, we find that as spreads widen, the price adjustment is less affected by the imbalance (spread = 1 is steeper than spread = 2)

!['Adj'](../data/images/adj.png)

In [19]:
w = np.linalg.matrix_power(B, 100)
spread_1 = w[0][:n_imbalance]
spread_2 = w[0][n_imbalance:]

fig = go.Figure()
fig.add_trace(go.Scatter(x=imb, y=spread_1, name='Spread = 1 tick adj'))
fig.add_trace(go.Scatter(x=imb, y=spread_2, name='Spread = 2 tick adj'))
fig.update_layout(title='Stationary Distribution', xaxis_title='Imbalance Bucket', yaxis_title='Frequency', height=700, width=800)
fig.show()

!['Counts'](../data/images/counts.png)

In [20]:
# lets add the calculated micro price to our time series
spreads = {0.01: 0,
           0.02: 10}

micro_price = []
for i in range(len(order_book)):
    dec = spreads.get(order_book.iloc[i]['Spread'])
    unit = order_book.iloc[i]['Imbalance_Bucket']
    micro = order_book.iloc[i]['Mid'] + G6[dec+unit]
    micro_price.append(micro)

order_book['Micro_Price'] = micro_price

In [23]:
bid = go.Scatter(x=order_book.index, y=order_book.Bid_1, mode='lines', line=dict(color='green'), name='Bid')
ask = go.Scatter(x=order_book.index, y=order_book.Ask_1, mode='lines', line=dict(color='red'), name='Ask')
mid = go.Scatter(x=order_book.index, y=order_book.Mid, mode='lines', line=dict(color='blue'), name='Mid')
wmid = go.Scatter(x=order_book.index, y=order_book.Weighted_Mid, mode='lines', line=dict(color='black'), name='W-Mid')
micro = go.Scatter(x=order_book.index, y=order_book.Micro_Price, mode='lines', line=dict(color='purple'), name='Micro')
traces = [bid, ask, mid, wmid, micro]
fig = go.Figure(data=traces)
fig.update_layout(title='MSFT Bid / Ask', xaxis_title='Time', yaxis_title='Price', width=1440, height=800, hovermode='x')
fig.show()

!['Micro'](../data/images/micro.png)